In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC # probability=True
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler 

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import make_pipeline

from sklearn.metrics import log_loss

pipe = [
    ('scaler', MaxAbsScaler()), 
    ('NB', BernoulliNB())
]
pipeline = Pipeline(pipe)

In [6]:
df_params = pd.read_csv('./Data/clean_train_values.csv', index_col = 0)

In [7]:
df_ID_Col = df_params['patient_id']
df_params = df_params.drop(['patient_id'], axis = 1)
df_params.head()

slope_of_peak_exercise_st_segment  resting_blood_pressure  chest_pain_type  \
0                                  1                     128                2   
1                                  2                     110                3   
2                                  1                     125                4   
3                                  1                     152                4   
4                                  3                     178                1   

   num_major_vessels  resting_ekg_results  serum_cholesterol_mg_per_dl  \
0                  0                    2                          308   
1                  0                    0                          214   
2                  3                    2                          304   
3                  0                    0                          223   
4                  0                    2                          270   

   oldpeak_eq_st_depression  age  max_heart_rate_achieved  thal_fixed_defect  \
0                       0.0   45                      170                  0   
1                       1.6   54                      158                  0   
2                       0.0   77                      162                  0   
3                       0.0   40                      181                  0   
4                       4.2   59                      145                  0   

   thal_normal  thal_reversible_defect  fast_bs_0  fast_bs_1  sex_0  sex_1  \
0            1                       0          1          0      0      1   
1            1                       0          1          0      1      0   
2            1                       0          1          0      0      1   
3            0                       1          1          0      0      1   
4            0                       1          1          0      0      1   

   exer_ang_0  exer_ang_1  
0           1           0  
1           1           0  
2           0           1  
3           1           0  
4           1           0

In [8]:
df_params.columns

Index(['slope_of_peak_exercise_st_segment', 'resting_blood_pressure',
       'chest_pain_type', 'num_major_vessels', 'resting_ekg_results',
       'serum_cholesterol_mg_per_dl', 'oldpeak_eq_st_depression', 'age',
       'max_heart_rate_achieved', 'thal_fixed_defect', 'thal_normal',
       'thal_reversible_defect', 'fast_bs_0', 'fast_bs_1', 'sex_0', 'sex_1',
       'exer_ang_0', 'exer_ang_1'],
      dtype='object')

In [9]:
df_labels = pd.read_csv('./Data/train_labels.csv')

In [10]:
df_labels.head()

patient_id  heart_disease_present
0     0z64un                      0
1     ryoo3j                      0
2     yt1s1x                      1
3     l2xjde                      1
4     oyt4ek                      0

alpha : float, optional (default=1.0)
Additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).

binarize : float or None, optional (default=0.0)
Threshold for binarizing (mapping to booleans) of sample features. If None, input is presumed to already consist of binary vectors.

fit_prior : boolean, optional (default=True)
Whether to learn class prior probabilities or not. If false, a uniform prior will be used.

class_prior : array-like, size=[n_classes,], optional (default=None)
Prior probabilities of the classes. If specified the priors are not adjusted according to the data.

In [11]:
X_train, X_test, y_train, y_test = train_test_split( df_params, df_labels["heart_disease_present"], test_size=0.2, random_state=42, stratify = df_labels["heart_disease_present"])

In [12]:
scalers = [StandardScaler(), MaxAbsScaler(), MinMaxScaler()]

In [13]:
value_range = np.arange(0, 1.1, 0.1)
neighbor_range = np.arange(1,6,1)
svc_range = np.logspace(-3, 2, 6) 
#https://stackoverflow.com/questions/26337403/what-is-a-good-range-of-values-for-the-svm-svc-hyperparameters-to-be-explored
gbc_learn = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
#https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/
max_feat = ['sqrt','log2']
print(svc_range)

[1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02]


In [14]:
#Warning surpress
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [15]:
models = [
    ('NB', BernoulliNB()),
    ('KNN', KNeighborsClassifier()),
    ('SVC', SVC(probability = True)),
    ('DTC', DecisionTreeClassifier()),
    ('GBC', GradientBoostingClassifier()),
        ]
print(models[0][1])

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)


In [16]:
#Create Param Grids for each model
Parameter_Grid = [
    {
        'NB__alpha':value_range, 
        'NB__binarize':value_range,
        'NB__fit_prior':[True,False] 
    },
    {
        'KNN__n_neighbors':neighbor_range,
        'KNN__weights':['uniform','distance'],
        'KNN__algorithm':['ball_tree','kd_tree','brute'],
    },
    {
        'SVC__C':svc_range,
        'SVC__kernel':['linear','poly','rbf','sigmoid'],
        'SVC__gamma':svc_range,
    },
    {
        'DTC__criterion':['gini','entropy'],
        'DTC__max_features':max_feat,
    },
    {
        'GBC__loss':['deviance','exponential'],
        'GBC__max_features':max_feat,
        'GBC__learning_rate':gbc_learn
    } ]

In [17]:
#load test values
test_vals = pd.read_csv("./Data/train_values.csv")

In [18]:
df = test_vals.drop("patient_id",axis = 1)
df[['sex', 'thal', 'fasting_blood_sugar_gt_120_mg_per_dl','exercise_induced_angina']] = df[['sex', 'thal','fasting_blood_sugar_gt_120_mg_per_dl','exercise_induced_angina']].astype('category') 

In [19]:
df = pd.get_dummies(df, columns=["thal", "fasting_blood_sugar_gt_120_mg_per_dl", "sex", "exercise_induced_angina"], prefix=["thal", "fast_bs", "sex","exer_ang"])

In [20]:
print(df_labels.shape)
print(test_vals.shape)
print(df_labels.head(5))

(180, 2)
(180, 14)
  patient_id  heart_disease_present
0     0z64un                      0
1     ryoo3j                      0
2     yt1s1x                      1
3     l2xjde                      1
4     oyt4ek                      0


In [23]:
#Get all inputs in order to reuse later

def Classification_Search(models,scalers,label_column,training_values,testing_values,training_labels,testing_labels)    
    scores = []
    highScore = []
    #drop predictions if present
    if 'heart_disease_present' in df.columns:
        df.drop("patient_id",axis = 1)
    #Create GridSearch Pipeline
    for count,mod in enumerate(models):
        for sc in scalers:
            pipe = [
                ('scaler',sc),
                mod
            ]
            pipeline = Pipeline(pipe)
            #initiate gridsearch
            grid = GridSearchCV(pipeline, param_grid=Parameter_Grid[count], cv=5)
            grid.fit(X_train,y_train)
            predictions = grid.predict_proba(df)[:,1]
            test_vals['heart_disease_present'] = predictions
            loss = log_loss(df_labels["heart_disease_present"], test_vals['heart_disease_present'])

            scores.append((mod,('scaler',sc),loss))
            #save results based on scaler and model
            test_vals[['patient_id','heart_disease_present']].to_csv("./Data/" + str(mod[0])+ "_" + str(sc)[0:str(sc).find("(")] + "_predictions.csv", index = False)

    Score_df = pd.DataFrame(scores)
    Score_df.head()

    Score_df.columns = ['Model','Scaler','metric']
    Score_df = Score_df.sort_values("metric")
    Score_df.reset_index()
    print(Score_df.head())
    return Score_df
    

[]
[(('NB', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), 0.5749504491816851), (('NB', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)), ('scaler', MaxAbsScaler(copy=True)), 0.6985122362290153), (('NB', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), 0.5699018031221393), (('KNN', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), 0.702416892481865), (('KNN', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')), ('scaler', MaxAbsScaler(copy=True)), 0.6812138677488245), (('KNN', KNeighborsC

In [26]:
scores[-1][2]

0.40228336987635493

In [28]:
Score_df = pd.DataFrame(scores)
Score_df.head()

0  \
0  (NB, BernoulliNB(alpha=1.0, binarize=0.0, clas...   
1  (NB, BernoulliNB(alpha=1.0, binarize=0.0, clas...   
2  (NB, BernoulliNB(alpha=1.0, binarize=0.0, clas...   
3  (KNN, KNeighborsClassifier(algorithm='auto', l...   
4  (KNN, KNeighborsClassifier(algorithm='auto', l...   

                                                   1         2  
0  (scaler, StandardScaler(copy=True, with_mean=T...  0.574950  
1                  (scaler, MaxAbsScaler(copy=True))  0.698512  
2  (scaler, MinMaxScaler(copy=True, feature_range...  0.569902  
3  (scaler, StandardScaler(copy=True, with_mean=T...  0.702417  
4                  (scaler, MaxAbsScaler(copy=True))  0.681214

In [30]:
Score_df.columns = ['Model',
                   'Scaler','log_loss']
Score_df = Score_df.sort_values("log_loss")
Score_df.reset_index()

index                                              Model  \
0       6  (SVC, SVC(C=1.0, cache_size=200, class_weight=...   
1       8  (SVC, SVC(C=1.0, cache_size=200, class_weight=...   
2      14  (GBC, GradientBoostingClassifier(criterion='fr...   
3       7  (SVC, SVC(C=1.0, cache_size=200, class_weight=...   
4      13  (GBC, GradientBoostingClassifier(criterion='fr...   
5      12  (GBC, GradientBoostingClassifier(criterion='fr...   
6       2  (NB, BernoulliNB(alpha=1.0, binarize=0.0, clas...   
7       0  (NB, BernoulliNB(alpha=1.0, binarize=0.0, clas...   
8       4  (KNN, KNeighborsClassifier(algorithm='auto', l...   
9       1  (NB, BernoulliNB(alpha=1.0, binarize=0.0, clas...   
10      5  (KNN, KNeighborsClassifier(algorithm='auto', l...   
11      3  (KNN, KNeighborsClassifier(algorithm='auto', l...   
12     10  (DTC, DecisionTreeClassifier(class_weight=None...   
13      9  (DTC, DecisionTreeClassifier(class_weight=None...   
14     11  (DTC, DecisionTreeClassifier(class_weight=None...   

                                               Scaler  log_loss  
0   (scaler, StandardScaler(copy=True, with_mean=T...  0.383189  
1   (scaler, MinMaxScaler(copy=True, feature_range...  0.400784  
2   (scaler, MinMaxScaler(copy=True, feature_range...  0.402283  
3                   (scaler, MaxAbsScaler(copy=True))  0.412782  
4                   (scaler, MaxAbsScaler(copy=True))  0.427833  
5   (scaler, StandardScaler(copy=True, with_mean=T...  0.429925  
6   (scaler, MinMaxScaler(copy=True, feature_range...  0.569902  
7   (scaler, StandardScaler(copy=True, with_mean=T...  0.574950  
8                   (scaler, MaxAbsScaler(copy=True))  0.681214  
9                   (scaler, MaxAbsScaler(copy=True))  0.698512  
10  (scaler, MinMaxScaler(copy=True, feature_range...  0.700757  
11  (scaler, StandardScaler(copy=True, with_mean=T...  0.702417  
12                  (scaler, MaxAbsScaler(copy=True))  1.343188  
13  (scaler, StandardScaler(copy=True, with_mean=T...  1.726965  
14  (scaler, MinMaxScaler(copy=True, feature_range...  1.726970

Top values were SVC with standard scaler, GBC with MinMaxScaler, GBC with StandardScaler

In [ ]:
Score_df.columns = ['Model','Scaler','log_loss']
print(Score_df.columns)

print(Score_df[['Model','Scaler']].iloc[0:3])

Row_list = []
for index, rows in Score_df[['Model','Scaler']].iloc[0:3].iterrows(): 
    # Create list for the current row 
    my_list =[rows.Model, rows.Scaler] 
      
    # append the list to the final list 
    Row_list.append(my_list) 
print(Row_list)

Parameter_Grid = [
    {
        'SVC__C':svc_range,
        'SVC__kernel':['linear','poly','rbf','sigmoid'],
        'SVC__gamma':svc_range,
    },
    {
        'GBC__loss':['deviance','exponential'],
        'GBC__max_features':max_feat,
        'GBC__learning_rate':gbc_learn
    },
{
        'GBC__loss':['deviance','exponential'],
        'GBC__max_features':max_feat,
        'GBC__learning_rate':gbc_learn
    }]
models = [
    ('SVC', SVC(probability = True)),
    ('GBC', GradientBoostingClassifier()),
    ('GBC', GradientBoostingClassifier()),
        ]
scalers = [StandardScaler(), MinMaxScaler()]

Test_predictions = pd.read_csv("./Data/test_values.csv")

Test_predictions_id = pd.DataFrame(Test_predictions[['patient_id']])
df = Test_predictions.drop("patient_id",axis = 1)
df[['sex', 'thal', 'fasting_blood_sugar_gt_120_mg_per_dl','exercise_induced_angina']] = df[['sex', 'thal','fasting_blood_sugar_gt_120_mg_per_dl','exercise_induced_angina']].astype('category') 
df = pd.get_dummies(df, columns=["thal", "fasting_blood_sugar_gt_120_mg_per_dl", "sex", "exercise_induced_angina"], prefix=["thal", "fast_bs", "sex","exer_ang"])
df.head()
type(Test_predictions_id)

#initialize smaller param list
#Same as other grid search, but make predictions too
for count,mod in enumerate(models):
    for sc in scalers:
        pipe = [
            ('scaler',sc),
            mod
        ]
        pipeline = Pipeline(pipe)
        #initiate gridsearch
        grid = GridSearchCV(pipeline, param_grid=Parameter_Grid[count], cv=5)
        grid.fit(X_train,y_train)
        predictions = grid.predict_proba(df)[:,1]
        Test_predictions_id['heart_disease_present'] = predictions
        Test_predictions_id.to_csv("./Data/_Final" + str(mod[0])+ "_" + str(sc) + "_predictions.csv", index = False)

#ensemble model
SCV_df = pd.read_csv("./Data/_FinalSVC_StandardScaler(copy%3DTrue%2C%20with_mean%3DTrue%2C%20with_std%3DTrue)_predictions.csv")
GBC_SS_df = pd.read_csv("./Data/_FinalGBC_StandardScaler(copy%3DTrue%2C%20with_mean%3DTrue%2C%20with_std%3DTrue)_predictions.csv")
GBC_MM_df = pd.read_csv("./Data/_FinalGBC_MinMaxScaler(copy%3DTrue%2C%20feature_range%3D(0%2C%201))_predictions.csv")

In [ ]:
#All below are markdown so I don't accidentally run a test model

pipe = [
    ('scaler', MaxAbsScaler()), 
    ('NB', BernoulliNB())
]
pipeline = Pipeline(pipe)

parameters = {'NB__alpha':value_range, 
              'NB__binarize':value_range,
              'NB__fit_prior':[True,False]}

grid = GridSearchCV(pipeline, param_grid=parameters, cv=5)

grid.fit(X_train, y_train)

x_test = pd.read_csv("./Data/test_values.csv")

x_test.head()

df.head()

predictions = grid.predict_proba(df)[:,1]

predictions

x_test['heart_disease_present'] = predictions
x_test.head()

x_test[['patient_id','heart_disease_present']].to_csv("./Data/NB_predictions.csv")